<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">navigation</h3>

* [1. Library](#1)
* [2. Data reading](#2)
* [3. Get longitude and latitude user_location column](#3)
* [4. Data preprocessing adn Text cleaning](#4)
* [5. Simple sentiment analysis and Create wordcloud](#5)
* [6. geographical distribution of tweets](#6)

In [ ]:
 #This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>

# Library

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
from wordcloud import WordCloud
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import plotly.express as px

nltk.download('stopwords')


stop_words = set(stopwords.words("english"))
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')
default_tokenizer=RegexpTokenizer(r"\w+")

<a id="2"></a>
# Data reading

In [ ]:
df = pd.read_csv('../input/covid19-tweets/covid19_tweets.csv')
df.head()

In [ ]:
df = df.sample(n=2000)
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df_columns = df.columns.tolist()

In [ ]:
dataTypeSeries = df.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

In [ ]:
df[df_columns].isnull().sum()

<a id="3"></a>
# Get longitude and latitude user_location column

In [ ]:
def do_geocode(address):
    geopy = Nominatim(user_agent='http')
    return geopy.geocode(address,exactly_one=True, timeout=1)




In [ ]:
#Creating Geocoded Location column
df['GeocodedLocation']=df['user_location'].apply(lambda x: do_geocode(x) if x != None else None)

In [ ]:
#Create the Latitude Column
lat=[]
for i in df['GeocodedLocation']:
    if i== None:
        lat.append(None)
    else:
        lat.append(i.latitude)
df['Latitude']=lat
df['Latitude'].astype('float')

In [ ]:
#Create the Longitude Column
long=[]
for i in df['GeocodedLocation']:
    if i== None:
        long.append(None)
    else:
        long.append(i.longitude)
df['Longitude']=long
df['Longitude'].astype('float')

<a id="4"></a>
# Data preprocessing and Text cleaning 

In [ ]:
#df = df.drop(["user_location"],axis = 1)

In [ ]:
df.head()

In [ ]:
df['user_created'] =  pd.to_datetime(df['user_created'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date
df['day'] = pd.to_datetime(df['date']).dt.day
df = df.sort_values(by='date')


In [ ]:
 def clean_text(text, ):
        if text is not None:
        #exclusions = '|'.join(exclusions)
                text = re.sub(r'[0-9]+','',text)
                text =  text.lower()
                text = re.sub('-', '', text)
                text = re.sub('_', '', text)
                text = re.sub('@', '', text)
                text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
                #text = re.sub(r'\S*@\S*\s?', '', text, flags=re.MULTILINE)
        # Remove text between square brackets
                text =re.sub('\[[^]]*\]', '', text)
        # removes punctuation
                text = re.sub(r'[^\w\s]','',text)
                text = re.sub(r'\n',' ',text)
                text = re.sub(r'[0-9]+','',text)
                #text = re.sub(r'[0-9]+','',text)
        # strip html 
                p = re.compile(r'<.*?>')
                text = re.sub(r"\'ve", " have ", text)
                text = re.sub(r"can't", "cannot ", text)
                text = re.sub(r"n't", " not ", text)
                text = re.sub(r"I'm", "I am", text)
                text = re.sub(r" m ", " am ", text)
                text = re.sub(r"\'re", " are ", text)
                text = re.sub(r"\'d", " would ", text)
                text = re.sub(r"\'ll", " will ", text)
        
                text = p.sub('', text)

        def tokenize_text(text,tokenizer=default_tokenizer):
            token = default_tokenizer.tokenize(text)
            return token
        
        def remove_stopwords(text, stop_words=default_stopwords):
            tokens = [w for w in tokenize_text(text) if w not in stop_words]
            return ' '.join(tokens)

        def stem_text(text, stemmer=default_stemmer):
            tokens = tokenize_text(text)
            return ' '.join([stemmer.stem(t) for t in tokens])

        text = stem_text(text) # stemming
        text = remove_stopwords(text) # remove stopwords
        #text.strip(' ') # strip whitespaces again?

        return text

In [ ]:
df['user_name'] = df['user_name'].apply(clean_text)


In [ ]:
df['user_description'] = df['user_description'].replace(np.nan, '', regex=True)
df['user_description'] = df['user_description'].apply(clean_text)

In [ ]:
df['text'] = df['text'].apply(clean_text)

<a id="5"></a>
# Simple sentiment analysis and create world cloud

In [ ]:
tweet_text_list = df.text.tolist()
tweet_text_string = ''.join(tweet_text_list)

In [ ]:
wordcloud_ip = WordCloud(
                      background_color='black',
                      margin=3,
                      width=1800,
                      height=1400,
                      max_words=200
                     ).generate(tweet_text_string)

plt.figure( figsize=(20,10) )
plt.imshow(wordcloud_ip)

<a id="6"></a>
# Geographical distribution of tweets

In [ ]:
fig = px.scatter_geo(df,lat = df["Latitude"] ,
                     lon = df["Longitude"],
                     animation_frame="day",
                     hover_name="user_name",
                     animation_group="day",
                     #size="user_followers"
                    
                     )
fig.update_geos(projection_type="natural earth")
fig.show()